## CSRD Data Points 
- Processing CSRD datapoints into a list of dictionaries 

In [21]:
%pip install pandas langchain jq 

Note: you may need to restart the kernel to use updated packages.


In [33]:
import pandas as pd 
from dotenv import load_dotenv
import json 
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
import os

### Load/Parse Raw Excel Data to JSON 

In [35]:
sheets_df = pd.read_excel('data/esrs_datav3.xlsx', sheet_name=None)

In [38]:
sheets_data = {sheet_name: sheet.to_dict(orient="records") for sheet_name, sheet in sheets_df.items()}

file_path = "data/csrd_datapoints.json"

if not os.path.exists(file_path):
    with open(file_path, "w", encoding="utf-8") as json_file:
        json.dump(sheets_data, json_file, indent=4, ensure_ascii=False)
    print("Data saved to csrd_datapoints.json")
else: 
    print(f"File {file_path} already exists")


File data/csrd_datapoints.json already exists


In [39]:
data = None 

with open(file_path, "r", encoding="utf-8") as json_file:
    data = json.load(json_file)

In [40]:
json_string = json.dumps(data, indent=2)
print(json_string)

{
  "ESRS 2": [
    {
      "ID": "BP-1_01",
      "ESRS": "ESRS 2",
      "DR": "BP-1",
      "Paragraph": "5 a",
      "Related AR": NaN,
      "Name": "Basis for preparation of sustainability statement",
      "Data Type": "semi-narrative",
      "Conditional or alternative DP": NaN,
      "May \n[V]": NaN,
      "Appendix B - ESRS 2 \n(SFDR + PILLAR 3 + Benchmark + CL)": NaN,
      "DPs to be disclosed in case of phased-in [Appendix C - ESRS 1]\nUndertaking less than 750 employees": NaN,
      "Appendix C - ESRS 1\n[DPs subject to phased-in]\n": NaN
    },
    {
      "ID": "BP-1_02",
      "ESRS": "ESRS 2",
      "DR": "BP-1",
      "Paragraph": "5 b i",
      "Related AR": NaN,
      "Name": "Scope of consolidation of consolidated sustainability statement is same as for financial statements",
      "Data Type": "narrative",
      "Conditional or alternative DP": "Conditional",
      "May \n[V]": NaN,
      "Appendix B - ESRS 2 \n(SFDR + PILLAR 3 + Benchmark + CL)": NaN,
      "DP

In [41]:
doc = Document(
    page_content=json_string,
)

In [43]:
from langchain.embeddings import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings()
embedded_json = embeddings_model.embed_query(doc.page_content)

print(embedded_json)  # This is the numerical embedding


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}